# Pre-processing techniques

In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score

from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing, DisparateImpactRemover
from aif360.metrics import BinaryLabelDatasetMetric

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


In [2]:
df = pd.read_csv('../../data/final_features_df.csv')
df.head()

,Unnamed: 0,Age,Income,faves_pca0,faves_pca1,unfaves_pca0,unfaves_pca1,accessories,alcohol,animamted,...,Drama.2,Entertainment (Variety Shows),Factual,Learning,Music,News,Religion &amp; Ethics,Sport.1,Weather,Rating_bin
0,0,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,1,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,2,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,3,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,4,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0


In [3]:
df_0 = df.fillna(0)

In [4]:
privileged_groups = [{'Gender_M': 1}]
unprivileged_groups = [{'Gender_M': 0}]

In [5]:
df_0['Rating_bin'].value_counts()

0    31279
1     4841
Name: Rating_bin, dtype: int64

In [6]:
df_0.columns

Index(['Unnamed: 0', 'Age', 'Income', 'faves_pca0', 'faves_pca1',
       'unfaves_pca0', 'unfaves_pca1', 'accessories', 'alcohol', 'animamted',
       ...
       'Drama.2', 'Entertainment (Variety Shows)', 'Factual', 'Learning',
       'Music', 'News', 'Religion &amp; Ethics', 'Sport.1', 'Weather',
       'Rating_bin'],
      dtype='object', length=515)

In [7]:
aif360_df = StandardDataset(
    df = df_0.drop(['Gender_F', 'Unnamed: 0'], axis = 1),
    label_name = 'Rating_bin',
    protected_attribute_names = ['Gender_M'],
    favorable_classes = [0],
    privileged_classes = [df_0['Gender_M'], lambda x: x == 1]
)

In [8]:
df_train, df_test = aif360_df.split([0.85], shuffle=True, seed = 42)

In [9]:
X_train = df_train.features
y_train = df_train.labels

X_test = df_test.features
y_test = df_test.labels

In [10]:
metric_orig = BinaryLabelDatasetMetric(aif360_df, 
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups,
                                       )

print("Statistical Parity Difference between unprivileged and privileged groups in original dataset = %f" % metric_orig.statistical_parity_difference())

Statistical Parity Difference between unprivileged and privileged groups in original dataset = 0.018700


In [11]:
df_conv = aif360_df.convert_to_dataframe()[0]

In [12]:
df_train.convert_to_dataframe()[0].shape

(30702, 513)

In [13]:
X_train.shape

(30702, 512)

In [14]:
from aif360.sklearn.metrics import statistical_parity_difference
statistical_parity_difference(y_test, y_test, prot_attr= df_test.convert_to_dataframe()[0]['Gender_M'] == 1)

-0.0154855124915005

In [15]:
def statistical_parity(y, y_, Z, priv=None):
  if priv is None:
    values = np.unique(Z)
    counts = [np.mean(y[Z==z]) for z in values]
    priv = values[np.argmax(counts)]
    unpriv = [z for z in values if z != priv]
    print('Automatic priviledged value is', priv)
  else:
    unpriv = [z for z in values if z != priv]
  
  return np.array([np.mean([y_i for y_i, zi in zip(y_, Z) if zi == unp]) - np.mean([y_i for y_i, zi in zip(y_, Z) if zi == priv])
                   for unp in unpriv])


In [16]:
statistical_parity(y_test, y_test, df_test.convert_to_dataframe()[0]['Gender_M'] == 1)

Automatic priviledged value is True


array([-0.01548551])

It's interesting to note the differences between the ways to calculate Statistical Parity Difference. This notebook is left with all the previous cells intentionally.

## Baseline model: Decision Tree

In [17]:
clf = DecisionTreeClassifier(class_weight='balanced')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
confusion_matrix(y_test, y_pred)

              precision    recall  f1-score   support

         0.0       0.95      0.66      0.78      4738
         1.0       0.24      0.74      0.36       680

    accuracy                           0.67      5418
   macro avg       0.59      0.70      0.57      5418
weighted avg       0.86      0.67      0.72      5418



array([[3123, 1615],
       [ 176,  504]])

In [18]:
f1_score(y_test, y_pred)

0.36012861736334406

In [19]:
statistical_parity(y_test, y_pred, df_test.convert_to_dataframe()[0]['Gender_M'] == 1)

Automatic priviledged value is True


array([-0.05141679])

## Reweight

In [20]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)

In [21]:
RW.fit(aif360_df)
train_transf = RW.transform(df_train)
test_transf = RW.transform(df_test)

In [22]:
X_train = train_transf.features
y_train = train_transf.labels

X_test = test_transf.features
y_test = test_transf.labels

In [23]:
clf = DecisionTreeClassifier(class_weight='balanced')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
confusion_matrix(y_test, y_pred)

              precision    recall  f1-score   support

         0.0       0.95      0.66      0.78      4738
         1.0       0.24      0.74      0.36       680

    accuracy                           0.67      5418
   macro avg       0.59      0.70      0.57      5418
weighted avg       0.86      0.67      0.72      5418



array([[3123, 1615],
       [ 176,  504]])

In [24]:
f1_score(y_test, y_pred)

0.36012861736334406

In [25]:
statistical_parity(y_test, y_pred, test_transf.convert_to_dataframe()[0]['Gender_M'] == 1)

Automatic priviledged value is True


array([-0.05141679])

## Disparate Impact Remover

In [26]:
di = DisparateImpactRemover(repair_level = 1.0)
# dataset_transf_train = di.fit(aif360_df)

train_transf = di.fit_transform(df_train)
test_transf = di.fit_transform(df_test)

In [27]:
X_train = train_transf.features
y_train = train_transf.labels

X_test = test_transf.features
y_test = test_transf.labels

In [28]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
confusion_matrix(y_test, y_pred)

              precision    recall  f1-score   support

         0.0       0.88      1.00      0.93      4738
         1.0       0.58      0.04      0.08       680

    accuracy                           0.88      5418
   macro avg       0.73      0.52      0.51      5418
weighted avg       0.84      0.88      0.83      5418



array([[4717,   21],
       [ 651,   29]])

In [29]:
f1_score(y_test, y_pred)

0.07945205479452055

In [30]:
statistical_parity(y_test, y_pred, test_transf.convert_to_dataframe()[0]['Gender_M'] == 1)

Automatic priviledged value is True


array([0.0142572])

# Testing code from class

In [31]:
import itertools
from sklearn.base import BaseEstimator, TransformerMixin, MetaEstimatorMixin
from sklearn.utils import check_array
from sklearn.utils.validation import check_is_fitted
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
class CounterfactualPreProcessing(MetaEstimatorMixin, BaseEstimator):
    def __init__(self, estimator, sensitive_feature_ids):
        self.estimator = estimator
        self.sensitive_feature_ids = sensitive_feature_ids

    def fit(self, X, y):
      sensitive_columns = X[:, self.sensitive_feature_ids]
      self.unique_values = []
      for scol in sensitive_columns.T:
        self.unique_values+=[np.unique(scol).tolist()]
      
      X_rows = []
      y_rows = []
      
      for xi, yi in zip(X, y):
        for comb in itertools.product(*self.unique_values):
          comb = np.array(comb)
          xi[self.sensitive_feature_ids] = comb
          X_rows+=[xi.copy()]
          y_rows+= [yi]
              
      X_transf = np.array(X_rows)
      y_transf = np.array(y_rows)

      print(X_transf.shape, y_transf.shape)

      self.estimator.fit(X_transf, y_transf)
      return self
    
    def predict(self, X):
      return self.estimator.predict(X)
      
    def predict_proba(self, X):
      return self.estimator.predict_proba(X)


class Weighted(MetaEstimatorMixin, BaseEstimator):
  def __init__(self, estimator, sensitive_feature_ids):
    self.estimator = estimator
    self.sensitive_feature_ids = sensitive_feature_ids

  def fit(self, X, y):
    sensitive_columns = X[:, self.sensitive_feature_ids]
    sensitive_columns_y = np.append(X[:, self.sensitive_feature_ids], y[:, np.newaxis], axis=1)
    
    unique_feats = []
    for scol in sensitive_columns.T:
      unique_feats+=[np.unique(scol).tolist()]
    unique_y = np.unique(y_train).tolist()
    
    unique_feats_y = []
    for scol in sensitive_columns_y.T:
      unique_feats_y+=[np.unique(scol).tolist()]
      
    self.unique_values = []
    for scol in sensitive_columns.T:
      self.unique_values+=[np.unique(scol).tolist()]
    self.unique_values+=[np.unique(y).tolist()]
    
    sample_weight = np.ones(sensitive_columns_y.shape[0])
    for comb in itertools.product(*unique_feats):
      comb = np.array(comb)
      where = np.prod(sensitive_columns == comb, axis=1)!=0
      sample_weight[where] *= np.mean(np.prod(sensitive_columns == comb, axis=1))
    
    for comb in itertools.product(*[unique_y]):
      comb = np.array(comb)
      where = (y_train == comb[0])
      sample_weight[where] *= np.mean(y_train == comb[0])
      
    for comb in itertools.product(*unique_feats_y):
      comb = np.array(comb)
      where = np.prod(sensitive_columns_y == comb, axis=1)!=0
      sample_weight[where] /= np.mean(np.prod(sensitive_columns_y == comb, axis=1))
    
    self.estimator.fit(X, y, sample_weight=sample_weight)
    return self
  
  def predict(self, X):
    return self.estimator.predict(X)
    
  def predict_proba(self, X):
    return self.estimator.predict_proba(X)

In [34]:
X = df_0.drop(columns=['Rating_bin', 'Gender_F', 'Unnamed: 0'], axis = 1)
y = df_0['Rating_bin']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.85)

In [35]:
sensitive_feature_ids = X_train.columns.get_loc('Gender_M')
X_train_tr = X_train.to_numpy()
Z_train = X_train['Gender_M'] == 1

In [36]:
clf_fair_weighted = Weighted(LogisticRegression(class_weight=None, max_iter=10**3), sensitive_feature_ids=[sensitive_feature_ids])
clf_fair_weighted.fit(X_train_tr, y_train)

/tmp/ipykernel_130158/2375806845.py:44: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  sensitive_columns_y = np.append(X[:, self.sensitive_feature_ids], y[:, np.newaxis], axis=1)


Weighted(estimator=LogisticRegression(max_iter=1000),
         sensitive_feature_ids=[155])

In [37]:
y_test_ = clf_fair_weighted.predict(X_test.to_numpy())
Z_test = X_test['Gender_M']==1
print('Statistical parity', statistical_parity(y_test, y_test_, Z_test))
print('F1', f1_score(y_test, y_test_))

Automatic priviledged value is True
Statistical parity [0.01191151]
F1 0.05575757575757576


In [38]:
clf_fair_counter = CounterfactualPreProcessing(LogisticRegression(class_weight=None, max_iter=10**3), sensitive_feature_ids=[sensitive_feature_ids])
clf_fair_counter.fit(X_train_tr, y_train)

(61404, 512) (61404,)


CounterfactualPreProcessing(estimator=LogisticRegression(max_iter=1000),
                            sensitive_feature_ids=[155])

In [39]:
y_test_ = clf_fair_counter.predict(X_test.to_numpy())
Z_test = X_test['Gender_M'] == 1
print('Statistical parity', statistical_parity(y_test, y_test_, Z_test))
print('F1', f1_score(y_test, y_test_))

Automatic priviledged value is True
Statistical parity [0.01191151]
F1 0.05575757575757576


In [42]:
baseline_lr = LogisticRegression(class_weight='balanced', max_iter=10**3)
baseline_lr.fit(X_train, y_train)
y_pred = baseline_lr.predict(X_test)

print('Statistical parity', statistical_parity(y_test, y_pred, Z_test))
print('F1', f1_score(y_test, y_pred))

Automatic priviledged value is True
Statistical parity [-0.03088743]
F1 0.38822387576835976
